 # Classification des avis sur des vêtements de femmes vendus dans le e-commerce

- est ce que les avis que l'on a des vêtements est représentatif de la note qui est attribué ?

idées 
- visualisation de données : quels types de vêtements ont les notes les plus élevées ?
- nb d'avis donné selon l'âge des clients

In [2]:
import pandas as pd

## I. Import des données

In [3]:
data = pd.read_csv("Womens Clothing E-Commerce Reviews.csv", sep = ",")
data.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [5]:
nb_dpt = data.groupby("Department Name").count()
nb_dpt

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Class Name
Department Name,,,,,,,,,,
Bottoms,3799,3799,3799,3184,3662,3799,3799,3799,3799,3799
Dresses,6319,6319,6319,5372,6145,6319,6319,6319,6319,6319
Intimate,1735,1735,1735,1408,1653,1735,1735,1735,1735,1735
Jackets,1032,1032,1032,879,1002,1032,1032,1032,1032,1032
Tops,10468,10468,10468,8713,10048,10468,10468,10468,10468,10468
Trend,119,119,119,107,118,119,119,119,119,119


### Récupération des données pour notre étude

### Traitement de la casse

- suppression des valeurs manquantes => pas d'avis : inutile
- suppression des caractères spéciaux 
- suppression des majuscules
- suppression des mots vides
- lemmatisation 
- affiche du nombre de mots par étiquette grammaticale
- extraction des mots (groupes de mots) les plus fréquents

- wordcloud

## II. Pré-traitement des données

## III. Méthode de fouille de textes